In [1]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


Using TensorFlow backend.


In [2]:

# semilla aleatoria
seed = 7
numpy.random.seed(seed)
# cargamos los datos
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [3]:
# normalizand las entradas en un rango de 0 a 1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# Creacion del modelo
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu',
kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='relu', padding='same',
kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [5]:
# codificacion en caliente de salida
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]



In [6]:
# Compilamos el modelo
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [7]:
# Ajustamos el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 24s 477us/step - loss: 1.7310 - accuracy: 0.3723 - val_loss: 1.3614 - val_accuracy: 0.5068
Epoch 2/25
50000/50000 [==============================] - 18s 356us/step - loss: 1.3511 - accuracy: 0.5135 - val_loss: 1.2106 - val_accuracy: 0.5682
Epoch 3/25
50000/50000 [==============================] - 18s 354us/step - loss: 1.1663 - accuracy: 0.5839 - val_loss: 1.0813 - val_accuracy: 0.6163
Epoch 4/25
50000/50000 [==============================] - 18s 354us/step - loss: 1.0379 - accuracy: 0.6296 - val_loss: 1.0287 - val_accuracy: 0.6368
Epoch 5/25
50000/50000 [==============================] - 18s 355us/step - loss: 0.9359 - accuracy: 0.6683 - val_loss: 0.9706 - val_accuracy: 0.6588
Epoch 6/25
50000/50000 [==============================] - 18s 355us/step - loss: 0.8530 - accuracy: 0.6973 - val_loss: 0.9393 - val_accuracy: 0.6721
Epoch 7/25
50000/50000 [==============================] 

In [8]:
# Final evaluacion del modelo
scores = model.evaluate(X_test, y_test, verbose=0)
print("Precision: %.2f%%" % (scores[1]*100))

Precision: 70.54%
